In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [23]:
import pandas as pd
df=pd.DataFrame()
train_df = pd.read_csv("D:/ayhanSelimSoftwareOrganization/data/allSentimentBertData.csv",sep=";")
train_df=train_df[:1000]
train_df

C:\Users\lab3_ayhanakkaya\AppData\Local\Temp\ipykernel_9448\2948834764.py:3: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("D:/ayhanSelimSoftwareOrganization/data/allSentimentBertData.csv",sep=";")


,category,date,clean_text,time_period,id,author,created_utc,link_id,parent_id,subreddit,subreddit_id,score,sentiment_score,sentiment
0,vaccine,08/01/2020,hurts less to put the needle in quickly not su...,"('01/01/2020', '01/02/2020')",fdkp8sp,Runnrgirl,1578508295.0,t3_ccmp3v,t3_ccmp3v,Vaccine,t5_36bc6,1.0,-0.8106,negative
1,vaccine,08/01/2020,tb test,"('01/01/2020', '01/02/2020')",fdkp9bu,Runnrgirl,1578508304.0,t3_ccmp3v,t1_f5ra6rw,Vaccine,t5_36bc6,1.0,0.0000,neutral
2,vaccine,19/01/2020,my body my choice,"('01/01/2020', '01/02/2020')",ff0r7wz,yinzer111,1579471539.0,t3_e85iev,t3_e85iev,Vaccine,t5_36bc6,1.0,0.0000,neutral
3,vaccine,30/01/2020,they say if you have auto immune your child ha...,"('01/01/2020', '01/02/2020')",ffz15b5,justfloatinon,1580360624.0,t3_e2nr8x,t3_e2nr8x,Vaccine,t5_36bc6,1.0,0.6597,positive
4,vaccine,30/01/2020,i think the future would be lower adjuvant vac...,"('01/01/2020', '01/02/2020')",ffzmww8,reallythinkso,1580386424.0,t3_e2nr8x,t1_ffz15b5,Vaccine,t5_36bc6,1.0,0.3291,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,vaccine,02/01/2020,wait is crying now considered a vaccine injury,"('01/01/2020', '01/02/2020')",fcvauw3,hBoBh,1577998248.0,t3_ej3zrk,t3_ej3zrk,vaxxhappened,t5_3p3pv,46.0,-0.7096,negative
996,vaccine,02/01/2020,i saw this on start mandatory vaccinations on ...,"('01/01/2020', '01/02/2020')",fcvayd2,HOLOCRUSH,1577998304.0,t3_ej3nvw,t3_ej3nvw,vaxxhappened,t5_3p3pv,13.0,0.4767,positive
997,vaccine,02/01/2020,no the mom doesnt want the umvaccinated child ...,"('01/01/2020', '01/02/2020')",fcvazgf,Scally59,1577998322.0,t3_eixdwq,t1_fcv7cg5,vaxxhappened,t5_3p3pv,19.0,-0.9010,negative
998,vaccine,02/01/2020,dunno why people think non gmo food is any bet...,"('01/01/2020', '01/02/2020')",fcvb59n,jandajanda,1577998417.0,t3_eivjmp,t3_eivjmp,vaxxhappened,t5_3p3pv,2.0,-0.0516,negative


In [24]:
df[["text","sentiment"]]=train_df[["clean_text","sentiment"]]

In [25]:
df=df.dropna()

In [26]:
df['sentiment'].value_counts()

negative    404
positive    377
neutral     219
Name: sentiment, dtype: int64

In [27]:
possible_labels = df.sentiment.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'negative': 0, 'neutral': 1, 'positive': 2}

In [28]:
df['label'] = df.sentiment.replace(label_dict)

In [29]:
df.head()

,text,sentiment,label
0,hurts less to put the needle in quickly not su...,negative,0
1,tb test,neutral,1
2,my body my choice,neutral,1
3,they say if you have auto immune your child ha...,positive,2
4,i think the future would be lower adjuvant vac...,positive,2


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [31]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [32]:
df.groupby(['sentiment', 'label', 'data_type']).count()

text
sentiment label data_type      
negative  0     train       343
                val          61
neutral   1     train       186
                val          33
positive  2     train       321
                val          56

In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

SSLError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/vocab.txt (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1122)')))

In [26]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\dlemin\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [27]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [28]:
len(dataset_train), len(dataset_val)

(519176, 91620)

In [29]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [32]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [33]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [34]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [36]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)